In [1]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from transformers import CLIPProcessor, CLIPModel, ViTImageProcessor, ViTModel
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
import pickle
import time
from xgboost import XGBClassifier
import pandas as pd

In [2]:
torch.cuda.empty_cache()

In [2]:
# Paths
METADATA_PATH = r'/scratch/vjh9526/bdml_2025/project/datasets/THINGS-EEG/image_set/image_metadata.npy'
TRAIN_DIR = r'/scratch/vjh9526/bdml_2025/project/datasets/THINGS-EEG/image_set/training_images'
TEST_DIR = r'/scratch/vjh9526/bdml_2025/project/datasets/THINGS-EEG/image_set/test_images'
RESULTS_PATH = r'/scratch/vjh9526/bdml_2025/project/code/classifiers/natural_modes/results_new.pkl'
FEATURES_DIR = r'/scratch/vjh9526/bdml_2025/project/code/classifiers/natural_modes/features'
THINGS_MAP_PATH = r"/scratch/vjh9526/bdml_2025/project/code/classifiers/natural_modes/category27_top-down.tsv"
BATCH_SIZE = 256

os.makedirs(FEATURES_DIR, exist_ok=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [3]:
# your existing function
def load_image_labels(metadata_path: str, things_map_path: str):
    """Load image labels from metadata file and map to high-level THINGS concepts."""
    print(f"[INFO] Loading image labels from {metadata_path}")
    print(f"[INFO] Loading high-level image labels from {things_map_path}")
    meta = np.load(metadata_path, allow_pickle=True).item()
    things_map = pd.read_csv(things_map_path, delimiter="\t")
    files = meta['train_img_files']
    concepts = meta['train_img_concepts']
    things_concepts = meta['train_img_concepts_THINGS']
    
    path_to_label = {}
    for things_concept, concept, fname in zip(things_concepts, concepts, files):
        idx = int(things_concept.split("_")[0]) - 1
        row = things_map.iloc[idx]
        # if any 1 in row, pick its column name; else 'miscellaneous'
        if (row == 0).all():
            high_concept = 'miscellaneous'
        else:
            high_concept = str(things_map.columns[row == 1][0])
        path_key = os.path.join(concept, fname)
        path_to_label[path_key] = high_concept
        
    return path_to_label


# 2) Build high-level labels mapping
path_to_label = load_image_labels(METADATA_PATH, THINGS_MAP_PATH)

# 3) Extract raw file lists & fine-grained concepts
meta      = np.load(METADATA_PATH, allow_pickle=True).item()
files     = meta['train_img_files']        # e.g. ["img1.png", "img2.png", …]
concepts  = meta['train_img_concepts']     # e.g. ["cat", "dog", …]

# 4) Build the list of high-level labels
labels_high = [
    path_to_label[os.path.join(concept, fname)]
    for concept, fname in zip(concepts, files)
]

# 5) Encode high-level labels to integers
unique_high = sorted(set(labels_high))
high_to_idx = {lbl: i for i, lbl in enumerate(unique_high)}
y_high = np.array([high_to_idx[lbl] for lbl in labels_high])

# 6) Stratified train/val split on high-level labels
train_files, val_files, train_concepts, val_concepts, y_train, y_val = train_test_split(
    files, concepts, y_high,
    test_size=0.2,
    random_state=42,
    stratify=y_high
)

# 7) Transforms (unchanged)
def get_transforms():
    train_tfm = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    val_tfm = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    return train_tfm, val_tfm

train_tfm, val_tfm = get_transforms()

# 8) Dataset & DataLoaders (now using y_train, y_val of high-level labels)
class ImageDataset(Dataset):
    def __init__(self, base_dir, concepts, files, labels, transform):
        self.base_dir = base_dir
        self.concepts = concepts
        self.files    = files
        self.labels   = labels
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        concept = self.concepts[idx]
        fname   = self.files[idx]
        path    = os.path.join(self.base_dir, concept, fname)
        img     = Image.open(path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, self.labels[idx]

train_dataset = ImageDataset(
    TRAIN_DIR, train_concepts, train_files, y_train, train_tfm
)
val_dataset = ImageDataset(
    TRAIN_DIR, val_concepts, val_files, y_val, val_tfm
)

train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE,
    shuffle=True, num_workers=4
)
val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE,
    shuffle=False, num_workers=4
)


[INFO] Loading image labels from /scratch/vjh9526/bdml_2025/project/datasets/THINGS-EEG/image_set/image_metadata.npy
[INFO] Loading high-level image labels from /scratch/vjh9526/bdml_2025/project/code/classifiers/natural_modes/category27_top-down.tsv


In [5]:
# ------------------- Extractor Definitions -------------------
extractor_configs = [
    ('resnet50', lambda: models.resnet50(pretrained=True).eval().to(device), None),
    # ('vgg19',   lambda: (lambda m: setattr(m,'classifier',torch.nn.Sequential(*list(m.classifier.children())[:-3])) or m)(models.vgg19(pretrained=True)).eval().to(device), None),
    ('clip',    None, (CLIPProcessor.from_pretrained, CLIPModel.from_pretrained, 'openai/clip-vit-base-patch32')),
    ('vit',     None, (ViTImageProcessor.from_pretrained, ViTModel.from_pretrained, 'google/vit-base-patch16-224-in21k'))
]

In [6]:
# ------------------- Feature Extraction -------------------
for name, cnn_constructor, hf_config in extractor_configs:
    print(f"\n--- Extracting features with {name} ---")
    start_time = time.time()

    # Instantiate model
    if cnn_constructor:
        model = cnn_constructor()
        model.fc = torch.nn.Identity()
        def extract_batch(imgs):
            return model(imgs.to(device))
    else:
        proc_cls, model_cls, model_id = hf_config
        processor = proc_cls(model_id)
        model = model_cls(model_id).eval().to(device)
        if name == 'clip':
            def extract_batch(imgs):
                pil = [transforms.ToPILImage()(img) for img in imgs]
                inputs = processor(images=pil, return_tensors='pt', padding=True).to(device)
                return model.get_image_features(**inputs)
        else:  # vit
            def extract_batch(imgs):
                pil = [transforms.ToPILImage()(img) for img in imgs]
                inputs = processor(images=pil, return_tensors='pt', padding=True).to(device)
                out = model(**inputs)
                return out.last_hidden_state[:,0,:]

    # Loop splits
    for split, loader in [('train', train_loader), ('val', val_loader)]:
        all_feats, all_labels = [], []
        for i, (imgs, lbls) in enumerate(loader,1):
            t0 = time.time()
            with torch.no_grad(): feats = extract_batch(imgs)
            feats_cpu = feats.cpu().numpy()
            all_feats.append(feats_cpu)
            all_labels.append(lbls.numpy())
            print(f"[{name}-{split}] batch {i}/{len(loader)} in {time.time()-t0:.2f}s")
        X = np.concatenate(all_feats)
        y_arr = np.concatenate(all_labels)
        np.save(os.path.join(FEATURES_DIR, f"{name}_{split}_feats.npy"), X)
        np.save(os.path.join(FEATURES_DIR, f"{name}_{split}_labels.npy"), y_arr)
        del all_feats, all_labels, X, y_arr
        torch.cuda.empty_cache()

    # Remove model
    del model
    torch.cuda.empty_cache()
    print(f"Finished {name}, time: {time.time()-start_time:.2f}s")


--- Extracting features with resnet50 ---


/ext3/miniforge3/envs/eeg_env/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/ext3/miniforge3/envs/eeg_env/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[resnet50-train] batch 1/52 in 6.87s
[resnet50-train] batch 2/52 in 5.26s
[resnet50-train] batch 3/52 in 5.31s
[resnet50-train] batch 4/52 in 5.01s
[resnet50-train] batch 5/52 in 4.30s
[resnet50-train] batch 6/52 in 4.08s
[resnet50-train] batch 7/52 in 3.71s
[resnet50-train] batch 8/52 in 4.06s
[resnet50-train] batch 9/52 in 3.95s
[resnet50-train] batch 10/52 in 3.97s
[resnet50-train] batch 11/52 in 3.63s
[resnet50-train] batch 12/52 in 3.90s
[resnet50-train] batch 13/52 in 3.70s
[resnet50-train] batch 14/52 in 3.95s
[resnet50-train] batch 15/52 in 3.59s
[resnet50-train] batch 16/52 in 3.96s
[resnet50-train] batch 17/52 in 4.05s
[resnet50-train] batch 18/52 in 3.82s
[resnet50-train] batch 19/52 in 3.72s
[resnet50-train] batch 20/52 in 3.85s
[resnet50-train] batch 21/52 in 4.05s
[resnet50-train] batch 22/52 in 4.05s
[resnet50-train] batch 23/52 in 4.00s
[resnet50-train] batch 24/52 in 3.85s
[resnet50-train] batch 25/52 in 3.12s
[resnet50-train] batch 26/52 in 3.06s
[resnet50-train] batc

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Finished resnet50, time: 253.72s

--- Extracting features with clip ---
[clip-train] batch 1/52 in 3.25s
[clip-train] batch 2/52 in 3.87s
[clip-train] batch 3/52 in 3.19s
[clip-train] batch 4/52 in 3.85s
[clip-train] batch 5/52 in 2.32s
[clip-train] batch 6/52 in 2.78s
[clip-train] batch 7/52 in 3.30s
[clip-train] batch 8/52 in 3.96s
[clip-train] batch 9/52 in 3.27s
[clip-train] batch 10/52 in 3.44s
[clip-train] batch 11/52 in 2.76s
[clip-train] batch 12/52 in 2.68s
[clip-train] batch 13/52 in 3.10s
[clip-train] batch 14/52 in 3.43s
[clip-train] batch 15/52 in 3.83s
[clip-train] batch 16/52 in 2.68s
[clip-train] batch 17/52 in 3.62s
[clip-train] batch 18/52 in 3.09s
[clip-train] batch 19/52 in 2.42s
[clip-train] batch 20/52 in 3.46s
[clip-train] batch 21/52 in 2.60s
[clip-train] batch 22/52 in 3.02s
[clip-train] batch 23/52 in 2.92s
[clip-train] batch 24/52 in 2.50s
[clip-train] batch 25/52 in 3.02s
[clip-train] batch 26/52 in 3.40s
[clip-train] batch 27/52 in 3.17s
[clip-train] batch 

/ext3/miniforge3/envs/eeg_env/lib/python3.11/site-packages/transformers/image_processing_utils.py:42: UserWarning: The following named arguments are not valid for `ViTImageProcessor.preprocess` and were ignored: 'padding'
  return self.preprocess(images, **kwargs)


[vit-train] batch 1/52 in 10.82s
[vit-train] batch 2/52 in 7.40s
[vit-train] batch 3/52 in 5.94s
[vit-train] batch 4/52 in 6.19s
[vit-train] batch 5/52 in 5.59s
[vit-train] batch 6/52 in 5.98s
[vit-train] batch 7/52 in 5.41s
[vit-train] batch 8/52 in 5.48s
[vit-train] batch 9/52 in 5.67s
[vit-train] batch 10/52 in 5.78s
[vit-train] batch 11/52 in 5.78s
[vit-train] batch 12/52 in 6.51s
[vit-train] batch 13/52 in 5.76s
[vit-train] batch 14/52 in 5.66s
[vit-train] batch 15/52 in 5.87s
[vit-train] batch 16/52 in 5.66s
[vit-train] batch 17/52 in 5.70s
[vit-train] batch 18/52 in 5.79s
[vit-train] batch 19/52 in 5.75s
[vit-train] batch 20/52 in 5.55s
[vit-train] batch 21/52 in 5.78s
[vit-train] batch 22/52 in 5.42s
[vit-train] batch 23/52 in 5.55s
[vit-train] batch 24/52 in 5.69s
[vit-train] batch 25/52 in 6.40s
[vit-train] batch 26/52 in 5.91s
[vit-train] batch 27/52 in 5.59s
[vit-train] batch 28/52 in 5.91s
[vit-train] batch 29/52 in 5.85s
[vit-train] batch 30/52 in 6.77s
[vit-train] batch 

In [7]:
import gc
gc.collect()

0

In [9]:
# ------------------- Classifier Training -------------------
from classifiers_eeg_embeds import SoftmaxClassifier, MLPClassifier, train_pytorch_model
from sklearn.tree           import DecisionTreeClassifier
from sklearn.linear_model  import LogisticRegression
from sklearn.naive_bayes   import GaussianNB
from sklearn.neighbors     import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics       import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix
)
import time

results = {}

for name, _, _ in extractor_configs:
    print(f"\n--- Training classifiers on {name} features ---")
    X_tr = np.load(os.path.join(FEATURES_DIR, f"{name}_train_feats.npy"))
    y_tr = np.load(os.path.join(FEATURES_DIR, f"{name}_train_labels.npy"))
    X_val = np.load(os.path.join(FEATURES_DIR, f"{name}_val_feats.npy"))
    y_val = np.load(os.path.join(FEATURES_DIR, f"{name}_val_labels.npy"))

    # scale
    scaler = StandardScaler()
    X_tr = scaler.fit_transform(X_tr)
    X_val = scaler.transform(X_val)

    # optional PCA for resnet50
    if name == "resnet50":
        pca = PCA(n_components=512, whiten=True, random_state=42)
        X_tr = pca.fit_transform(X_tr)
        X_val = pca.transform(X_val)

    num_classes = len(np.unique(y_tr))
    input_dim   = X_tr.shape[1]
    results[name] = {}

    print(f"Shape of training data is {X_tr.shape}")
    # --- scikit‑learn classifiers ---
    for clf_name, clf in {
        'dt'      : DecisionTreeClassifier(random_state=42),
        'logistic': LogisticRegression(
                         multi_class='multinomial',
                         max_iter=1000,
                         random_state=42
                     ),
        'nb'      : GaussianNB(),
        'knn'     : KNeighborsClassifier(n_neighbors=5)
    }.items():
        print(f"Training {clf_name} classifier...")
        t0 = time.time()
        clf.fit(X_tr, y_tr)
        preds = clf.predict(X_val)
        acc       = accuracy_score(y_val, preds)
        prec      = precision_score(y_val, preds, average='weighted', zero_division=0)
        rec       = recall_score(y_val, preds, average='weighted', zero_division=0)
        f1        = f1_score(y_val, preds, average='weighted', zero_division=0)
        print(f"[{name}-{clf_name}] acc={acc:.4f} precision={prec:.4f} "
              f"recall={rec:.4f} f1={f1:.4f} time={time.time()-t0:.2f}s")

        results[name][clf_name] = {
            'accuracy' : acc,
            'precision': prec,
            'recall'   : rec,
            'f1'       : f1,
            # 'report'   : classification_report(
            #                  y_val, preds,
            #                  target_names=unique_labels,
            #                  zero_division=0
            #              ),
            'cm'       : confusion_matrix(y_val, preds)
        }
        del clf

    # --- PyTorch classifiers ---
    for clf_name, Model in {
        'softmax': SoftmaxClassifier,
        'mlp'    : lambda in_dim, n_cls: MLPClassifier(in_dim, [512], n_cls)
    }.items():
        print(f"Training {clf_name} classifier...")
        # instantiate
        if clf_name == 'softmax':
            model = Model(input_dim, num_classes)
        else:
            model = Model(input_dim, num_classes)
        pt = train_pytorch_model(
            model, X_tr, X_val, y_tr, y_val,
            batch_size=BATCH_SIZE,
            device=device
        )
        preds = pt['y_pred']
        acc   = pt['accuracy']
        prec  = precision_score(y_val, preds, average='weighted', zero_division=0)
        rec   = recall_score(y_val, preds, average='weighted', zero_division=0)
        f1_   = f1_score(y_val, preds, average='weighted', zero_division=0)
        print(f"[{name}-{clf_name}] acc={acc:.4f} precision={prec:.4f} "
              f"recall={rec:.4f} f1={f1_:.4f}")

        results[name][clf_name] = {
            'accuracy'      : acc,
            'precision'     : prec,
            'recall'        : rec,
            'f1'            : f1_,
            'train_losses'  : pt['train_losses'],
            'val_accuracies': pt['val_accuracies'],
            # 'report'        : classification_report(
            #                       y_val, preds,
            #                       target_names=unique_labels,
            #                       zero_division=0
            #                   ),
            'cm'            : confusion_matrix(y_val, preds)
        }
        del model, pt

    # free memory before next extractor
    del X_tr, y_tr, X_val, y_val



--- Training classifiers on resnet50 features ---
Shape of training data is (13232, 512)
Training dt classifier...
[resnet50-dt] acc=0.5154 precision=0.5245 recall=0.5154 f1=0.5195 time=19.30s
Training logistic classifier...


/ext3/miniforge3/envs/eeg_env/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


[resnet50-logistic] acc=0.7104 precision=0.7168 recall=0.7104 f1=0.7119 time=0.59s
Training nb classifier...
[resnet50-nb] acc=0.5837 precision=0.6473 recall=0.5837 f1=0.5964 time=0.16s
Training knn classifier...
[resnet50-knn] acc=0.7536 precision=0.7989 recall=0.7536 f1=0.7563 time=0.10s
Training softmax classifier...
(13232, 512)
Epoch 10, Loss: 1.3476, Accuracy: 0.6632
Epoch 20, Loss: 1.0097, Accuracy: 0.7122
Epoch 30, Loss: 0.8381, Accuracy: 0.7364
Epoch 40, Loss: 0.7249, Accuracy: 0.7542
Epoch 50, Loss: 0.6407, Accuracy: 0.7588
Epoch 60, Loss: 0.5788, Accuracy: 0.7609
[INFO] Early stopping at epoch 68
[INFO] Best accuracy: 0.7633 at epoch 58
[resnet50-softmax] acc=0.7633 precision=0.7598 recall=0.7627 f1=0.7601
Training mlp classifier...
(13232, 512)
Epoch 10, Loss: 0.0643, Accuracy: 0.7993
Epoch 20, Loss: 0.0127, Accuracy: 0.7950
Epoch 30, Loss: 0.0053, Accuracy: 0.8011
[INFO] Early stopping at epoch 34
[INFO] Best accuracy: 0.8050 at epoch 24
[resnet50-mlp] acc=0.8050 precision

/ext3/miniforge3/envs/eeg_env/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


[clip-logistic] acc=0.5659 precision=0.5534 recall=0.5659 f1=0.5529 time=3.27s
Training nb classifier...
[clip-nb] acc=0.2113 precision=0.5532 recall=0.2113 f1=0.2036 time=0.13s
Training knn classifier...
[clip-knn] acc=0.5299 precision=0.5068 recall=0.5299 f1=0.5001 time=0.12s
Training softmax classifier...
(13232, 512)
Epoch 10, Loss: 1.8765, Accuracy: 0.5163
Epoch 20, Loss: 1.5658, Accuracy: 0.5517
Epoch 30, Loss: 1.3510, Accuracy: 0.5862
Epoch 40, Loss: 1.2154, Accuracy: 0.6010
Epoch 50, Loss: 1.1339, Accuracy: 0.6103
Epoch 60, Loss: 1.0849, Accuracy: 0.6137
[INFO] Early stopping at epoch 69
[INFO] Best accuracy: 0.6146 at epoch 59
[clip-softmax] acc=0.6146 precision=0.5779 recall=0.6134 f1=0.5801
Training mlp classifier...
(13232, 512)
Epoch 10, Loss: 0.7396, Accuracy: 0.6264
[INFO] Early stopping at epoch 14
[INFO] Best accuracy: 0.6333 at epoch 4
[clip-mlp] acc=0.6333 precision=0.5852 recall=0.6155 f1=0.5862

--- Training classifiers on vit features ---
Shape of training data is

/ext3/miniforge3/envs/eeg_env/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


[vit-logistic] acc=0.5003 precision=0.5096 recall=0.5003 f1=0.5014 time=165.12s
Training nb classifier...
[vit-nb] acc=0.2288 precision=0.4998 recall=0.2288 f1=0.2344 time=0.29s
Training knn classifier...
[vit-knn] acc=0.5082 precision=0.4725 recall=0.5082 f1=0.4770 time=0.14s
Training softmax classifier...
(13232, 768)
Epoch 10, Loss: 1.7942, Accuracy: 0.5006
Epoch 20, Loss: 1.4617, Accuracy: 0.5323
Epoch 30, Loss: 1.2357, Accuracy: 0.5571
Epoch 40, Loss: 1.0918, Accuracy: 0.5819
Epoch 50, Loss: 0.9980, Accuracy: 0.5846
Epoch 60, Loss: 0.9406, Accuracy: 0.5877
[INFO] Early stopping at epoch 65
[INFO] Best accuracy: 0.5892 at epoch 55
[vit-softmax] acc=0.5892 precision=0.5395 recall=0.5798 f1=0.5499
Training mlp classifier...
(13232, 768)
Epoch 10, Loss: 0.4012, Accuracy: 0.5985
[INFO] Early stopping at epoch 14
[INFO] Best accuracy: 0.6176 at epoch 4
[vit-mlp] acc=0.6176 precision=0.5590 recall=0.5955 f1=0.5620


In [10]:
# Save results
with open(RESULTS_PATH, 'wb') as f:
    pickle.dump(results, f)
print("\nAll done.")


All done.


In [ ]:
# # ------------------- Classifier Training -------------------
# classifiers = {
#     'lr': LogisticRegression(max_iter=1000),
#     'rf': RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42),
#     # 'gb': GradientBoostingClassifier(),
#     'xgb': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', n_jobs=-1, random_state=42),
#     'mlp': MLPClassifier(hidden_layer_sizes=(512,), max_iter=200),
#     'knn': KNeighborsClassifier(n_neighbors=5)
# }
# results = {}

# for name, _, _ in extractor_configs:
#     print(f"\n--- Training classifiers on {name} features ---")
#     X_tr = np.load(os.path.join(FEATURES_DIR, f"{name}_train_feats.npy"))
#     y_tr = np.load(os.path.join(FEATURES_DIR, f"{name}_train_labels.npy"))
#     X_val = np.load(os.path.join(FEATURES_DIR, f"{name}_val_feats.npy"))
#     y_val = np.load(os.path.join(FEATURES_DIR, f"{name}_val_labels.npy"))
#     # add standard scaler
#     from sklearn.preprocessing import StandardScaler
#     scaler = StandardScaler()
#     X_tr = scaler.fit_transform(X_tr)
#     X_val = scaler.transform(X_val)
#     if name == "resnet50":
#         # reduce dimensionality for resnet50 features
#         from sklearn.decomposition import PCA
#         pca = PCA(n_components=512, whiten=True, random_state=42)
#         pca.fit(X_tr)
#         X_tr = pca.transform(X_tr)
#         X_val = pca.transform(X_val)
#     results[name] = {}
#     for clf_name, clf in classifiers.items():
#         # add debug information
#         print(f"Training {clf_name} classifier...")
#         print(f"X_tr shape: {X_tr.shape}, y_tr shape: {y_tr.shape}")
#         t0 = time.time()
#         clf.fit(X_tr, y_tr)
#         preds = clf.predict(X_val)
#         acc = accuracy_score(y_val, preds)
#         # calculate precision, recall, f1-score
#         precision = precision_score(y_val, preds, average='weighted', zero_division=0)
#         recall = recall_score(y_val, preds, average='weighted', zero_division=0)
#         f1 = f1_score(y_val, preds, average='weighted', zero_division=0)
#         print(f"[{name}-{clf_name}] acc={acc:.4f} precision={precision:.4f} recall={recall:.4f} f1={f1:.4f} time={time.time()-t0:.2f}s")
#         results[name][clf_name] = {
#             'accuracy': acc,
#             'precision': precision,
#             'recall': recall,
#             'f1': f1,
#             'report': classification_report(y_val, preds, target_names=unique_labels, zero_division=0)
#         }
#         # print classification report
#         # print(results[name][clf_name]['report'])
#         del clf
#     del X_tr, y_tr, X_val, y_val


In [ ]:
# # Save results
# with open(RESULTS_PATH, 'wb') as f:
#     pickle.dump(results, f)
# print("\nAll done.")

In [ ]:
# # cluster and visualize, using t-SNE, the Resnet50, CLIP and ViT features
# import matplotlib.pyplot as plt
# from sklearn.manifold import TSNE
# from sklearn.decomposition import PCA
# import seaborn as sns
# import umap
# import umap.umap_ as umap
# import matplotlib.cm as cm
# import matplotlib.colors as mcolors
# import matplotlib.patches as mpatches
# import matplotlib.lines as mlines

# def plot_tsne(X, y, title, save_path):
#     tsne = TSNE(n_components=2, random_state=42, n_jobs=-1, verbose=1)
#     X_embedded = tsne.fit_transform(X)
#     plt.figure(figsize=(10, 10))
#     scatter = plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=y, cmap='tab10', alpha=0.5)
#     plt.title(title)
#     plt.colorbar(scatter, ticks=range(len(unique_labels)), label='Concepts')
#     plt.savefig(save_path)
#     plt.close()

# def plot_umap(X, y, title, save_path):
#     reducer = umap.UMAP(n_components=2, random_state=42, n_jobs=-1, verbose=1)
#     X_embedded = reducer.fit_transform(X)
#     plt.figure(figsize=(10, 10))
#     scatter = plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=y, cmap='tab10', alpha=0.5)
#     plt.title(title)
#     plt.colorbar(scatter, ticks=range(len(unique_labels)), label='Concepts')
#     plt.savefig(save_path)
#     plt.close()

# def plot_pca(X, y, title, save_path):
#     pca = PCA(n_components=2, random_state=42, whiten=True)
#     X_embedded = pca.fit_transform(X)
#     plt.figure(figsize=(10, 10))
#     scatter = plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=y, cmap='tab10', alpha=0.5)
#     plt.title(title)
#     plt.colorbar(scatter, ticks=range(len(unique_labels)), label='Concepts')
#     plt.savefig(save_path)
#     plt.close()

# # Plot and save t-SNE, UMAP and PCA for each feature set
# for name, _, _ in extractor_configs:
#     print(f"\n--- Plotting {name} features ---")
#     X_tr = np.load(os.path.join(FEATURES_DIR, f"{name}_train_feats.npy"))
#     y_tr = np.load(os.path.join(FEATURES_DIR, f"{name}_train_labels.npy"))
#     plot_tsne(X_tr, y_tr, f"{name} t-SNE", os.path.join(FEATURES_DIR, f"{name}_train_tsne.png"))
#     plot_umap(X_tr, y_tr, f"{name} UMAP", os.path.join(FEATURES_DIR, f"{name}_train_umap.png"))
#     plot_pca(X_tr, y_tr, f"{name} PCA", os.path.join(FEATURES_DIR, f"{name}_train_pca.png"))

In [ ]:
# # Now, run the visualizations again, but just for 10 classes, to make it easier to visualize
# # Select 10 cllasses randomly from the unique labels
# import random
# random.seed(42)

# num_classes = 50

# # load the features and labels for each feature set and then compute the unique labels, and select 10 classes randomly, and plot the t-SNE, UMAP and PCA for each feature set
# for name, _, _ in extractor_configs:
#     print(f"\n--- Plotting {name} features for {num_classes} classes ---")
#     X_tr = np.load(os.path.join(FEATURES_DIR, f"{name}_train_feats.npy"))
#     y_tr = np.load(os.path.join(FEATURES_DIR, f"{name}_train_labels.npy"))
#     unique_labels = np.unique(y_tr)
#     selected_labels = random.sample(list(unique_labels), num_classes)
#     mask = np.isin(y_tr, selected_labels)
#     X_tr = X_tr[mask]
#     y_tr = y_tr[mask]
#     plot_tsne(X_tr, y_tr, f"{name} t-SNE ({num_classes} classes)", os.path.join(FEATURES_DIR, f"{name}_train_tsne_{num_classes}_classes.png"))
#     plot_umap(X_tr, y_tr, f"{name} UMAP ({num_classes} classes)", os.path.join(FEATURES_DIR, f"{name}_train_umap_{num_classes}_classes.png"))
#     plot_pca(X_tr, y_tr, f"{name} PCA ({num_classes} classes)", os.path.join(FEATURES_DIR, f"{name}_train_pca_{num_classes}_classes.png"))